# Retriever

In [181]:
from database import DatabaseClient

class RetrieverClient:
    
    def __init__(self, folder_path):
        self.__database = DatabaseClient(folder_path)

    def __retrieve(self, text=None, image_path=None):
        if image_path == None and text == None:
            return []
        elif image_path != None and text != None:
            return self.__database.search_with_image(image_path) + self.database.search_with_text(text)
        elif text == None:
            return self.__database.search_with_image(image_path)
        elif image_path == None:
            return self.__database.search_with_text(text)
    
    def __organize_by_media_type(self, properties):
        response =  {
            'text': [],
            'image': []
        }
        for property in properties:
            if property["media_type"] == "text" and property not in response['text']:
                response['text'].append(property)
            if property["media_type"] == "image" and property not in response['image']:
                response.append(property)
        return response
    
    def search(self, text=None, image_path=None):
        return self.__organize_by_media_type(self.__retrieve(text, image_path))
    
    def close_database_connection(self):
        self.__database.close_connection()
        

# OfflineChat

In [171]:
# import ollama

# modelfile = """FROM phi3"""
# ollama.create(model="phi3", modelfile=modelfile)

{'status': 'success'}

In [209]:
import ollama

class OfflineChat:
    
    def __init__(self, model="phi3-rag"):
        self.__messages = []
        self.__model = model
    
    def __append_user_message(self, user_query, search_result):
        content = ""
        image_paths = []
        for text_properties in search_result['text']:
            content += text_properties['text'] + "\n"
        for image_properties in search_result['image']:
            image_paths.append(image_properties['path'])
        query_with_context = f"""Given Context: {content}
                                 Query: {user_query}"""
        self.__messages.append({"role": "user", "content": query_with_context, "images": image_paths})
    
    def append_assistant_message(self, content):
        self.__messages.append({"role": "assistant", "content" : content})  
         
    def get_assistant_response(self, user_text, search_result):
        """
        Usage:
            for chunk in get_assistant_response(...):
                assistant_response += chunk["message"]["content"]
                print(chunk["message"]["content"], end="", flush=True)
            offline_chat.append_assistant_message(assistant_response)
        """
        self.__append_user_message(user_text, search_result)
        return ollama.chat(self.__model, self.__messages, stream=True, options={"temperature":0})
    
    def get_history(self):
        return self.__messages


# OnlineChat

In [210]:
import google.generativeai as genai
from IPython.display import Image

class OnlineChat:
    
    def __init__(self, api_key):
        self.__chat = self.__initiate_chat(api_key=api_key)
         
    def __initiate_chat(self, api_key, model_name="gemini-1.5-flash-001"):
        system_instructions = "You are a RAG Chatbot and you will only respond using the information given to you and nothing else."
        safey_settings = [
            {
                "category": "HARM_CATEGORY_HARASSMENT",
                "threshold": "BLOCK_NONE",
            },
            {
                "category": "HARM_CATEGORY_HATE_SPEECH",
                "threshold": "BLOCK_NONE",
            },
            {
                "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
                "threshold": "BLOCK_NONE",
            },
            {
                "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
                "threshold": "BLOCK_NONE",
            },
        ]
        genai.configure(api_key=api_key)
        model = genai.GenerativeModel(model_name=model_name, 
                                      system_instruction=system_instructions, 
                                      safety_settings=safey_settings)
        return model.start_chat()
    
    def __create_user_message(self, user_text, search_result):
        content = ""
        images = []
        for text_properties in search_result['text']:
            content += text_properties['text'] + "\n"
        for image_properties in search_result['image']:
            images.append(Image(image_properties['path']))
        query_with_context = [f"Given Context: {content} \nQuery: {user_text}"] + images
        return query_with_context
    
    def get_assistant_response(self, user_text, search_result):
        """
        Usage:
            chat = OnlineChat(...)
            for chunk in chat.get_assistant_response(user_text, user_image_path):
                print(chunk.text, end="", flush=True)
        """
        return self.__chat.send_message(content=self.__create_user_message(user_text, search_result), stream=True)    
    
    def get_history(self):
        return self.__chat.history

# Tutorial for Integration

In [ ]:
from chat import OnlineChat, OfflineChat
from retriever import RetrieverClient

In [ ]:
def handle_offline_response(chat_client: OfflineChat, search_result, user_text=None):
    assistant_response = ""
    for chunk in chat_client.get_assistant_response(user_text=user_text, search_result=search_result):
        assistant_response += chunk["message"]["content"]
        print(chunk["message"]["content"], end="", flush=True)
        chat_client.append_assistant_message(assistant_response)
        
def handle_online_response(chat_client : OnlineChat, search_result, user_text=None):
    for chunk in chat_client.get_assistant_response(user_text=user_text, search_result=search_result):
        print(chunk.text, end="", flush=True)

In [ ]:
folder_path = r"C:\Users\Anush\Desktop\Christ\Specialization Project\Localinsight\documents"
mode = "online"
api_key = ""

user_query = "Why did Alice leave?"

In [123]:
retriever = RetrieverClient(folder_path=folder_path)
chat_client = OfflineChat() if mode == "offline" else OnlineChat(api_key=api_key)

In [218]:
search_result = retriever.search(text=user_query, image_path=None)

In [219]:
handle_offline_response(chat_client, search_result, user_query) if mode == "offline" \
else handle_online_response(chat_client, search_result, user_query)


{'text': [{'text': 'called out in a trembling voice to its children, “Come away, my dears! It’s high time you were all in bed!” On various pretexts they all moved off, and Alice was soon left alone.',
   'chunk_no': 147.0,
   'path': 'C:\\Users\\Anush\\Desktop\\Christ\\Specialization Project\\Localinsight\\text-only-test\\documents\\alice.txt',
   'media_type': 'text'},
  {'text': '“I wish I hadn’t mentioned Dinah!” she said to herself in a melancholy tone. “Nobody seems to like her, down here, and I’m sure she’s the best cat in the world! Oh, my dear Dinah! I wonder if I shall ever see you any more!” And here poor Alice began to cry again, for she felt very lonely and',
   'media_type': 'text',
   'path': 'C:\\Users\\Anush\\Desktop\\Christ\\Specialization Project\\Localinsight\\text-only-test\\documents\\alice.txt',
   'chunk_no': 148.0},
  {'text': '“Come, there’s no use in crying like that!” said Alice to herself, rather sharply; “I advise you to leave off this minute!” She generall

In [220]:
message_history = chat_client.get_history()

Alice was left alone because her mother called her and her siblings to bed. 
